### 웹캠으로 사람 수 세고, 상황 보고서 작성하는 웹앱
- 얼굴 인식으로 사람 수 자동 감지
- 사람 수가 바뀌면 OpenAI API로 상황 설명 자동 생성
- Flask 웹페이지에 웹캠 화면 + 사람 수 + 상황 설명 표시

In [1]:
from flask import Flask, render_template, Response
import cv2
import mediapipe as mp
import numpy as np
from PIL import ImageFont, ImageDraw, Image
from openai import OpenAI
import os
from dotenv import load_dotenv

# 한글 폰트 경로 
font_path = "C:/Windows/Fonts/malgun.ttf"  # 맑은 고딕 폰트

# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

# 환경 변수 가져오기
API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=API_KEY)

app = Flask(__name__)

# MediaPipe Face Detection 초기화
mp_face = mp.solutions.face_detection
face_detection = mp_face.FaceDetection(model_selection=0, min_detection_confidence=0.5)

# 비디오 캡처
cap = cv2.VideoCapture(0)
# cap = cv2.VideoCapture(1)

# 현재 사람 수와 상황 설명
current_people = 0
last_report = "아직 아무도 없습니다."

# 폰트 경로 (윈도우 기준)
font_path = "C:/Windows/Fonts/malgun.ttf"

# PIL로 한글 텍스트 그리기
def draw_text_with_pil(img, text, position, font_size=30, color=(0, 0, 255)):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    font = ImageFont.truetype(font_path, font_size)
    draw.text(position, text, font=font, fill=color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

# 상황 설명 생성
def generate_report(people_count):
    prompt = f"지금 카메라에 {people_count}명이 감지되었습니다. 이 상황을 짧게 설명하는 한 문장을 만들어줘."
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

def gen_frames():
    global current_people, last_report

    while True:
        success, frame = cap.read()
        if not success:
            break

        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_detection.process(rgb_frame)

        previous_people = current_people
        current_people = 0

        if results.detections:
            current_people = len(results.detections)

        # 사람 수가 변하면 새로운 상황 보고서 요청
        if current_people != previous_people:
            try:
                last_report = generate_report(current_people)
            except Exception as e:
                last_report = f"오류 발생: {e}"

        # 사람 수, 상황 설명 표시
        frame = draw_text_with_pil(frame, f"사람 수: {current_people}", (10, 30), font_size=40)
        frame = draw_text_with_pil(frame, last_report, (10, 80), font_size=30, color=(0, 255, 0))

        ret, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()

        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/video_feed')
def video_feed():
    return Response(gen_frames(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [29/Apr/2025 16:34:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2025 16:34:48] "GET /video_feed HTTP/1.1" 200 -
